# **<span style="font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## Introduction

This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory).
- **PrOMMiS public GitHub Repository:** [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)
- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

<div style="text-align: center;">
    <img src="images\uky_flowsheet.png" width="1000"/>
</div>

## Step 1: Import the necessary tools

In [ ]:
# import main libraries
import pandas as pd
import numpy as np
import olca_ipc
import olca_schema as olca
import netolca
from typing import Union, List, Optional

from pyomo.environ import (units,value,)

import pubchempy as pcp

from pymatgen.core import Composition

from warnings import warn

# module to run the PrOMMiS model and extract data 
from src import prommis_LCA_data
# module to run convert the PrOMMiS model data to LCA data
from src import prommis_LCA_conversions

# modules to connect to openLCA
from src.create_olca_process.olca_utils import (
    initialize_client, print_database_summary, generate_id, find_entity_by_name
)
from src.create_olca_process.dataframe_utils import (
    validate_dataframe_columns, load_dataframe_from_csv, get_amount_from_dataframe_row
)
from src.create_olca_process.search_utils import (
    get_existing_processes, search_processes_by_keywords
)
from src.create_olca_process.user_interface import (
    show_process_selection_menu, get_yes_no_input, get_user_search_choice, 
    show_unit_selection_in_flow_menu, show_flow_process_selection_menu
)
from src.create_olca_process.process_manager import create_process_from_dataframe_with_selection
from src.create_olca_process.flow_manager import find_compatible_flow_pattern, search_flows_by_name
from src.create_olca_process.unit_manager import (
    get_existing_units, get_common_unit_suggestions, search_units_by_name, 
    find_units_in_flow, find_unit_in_flow_by_name, create_unit_from_dataframe_row
)

from src import prommis_LCA_conversions
from src import finalize_LCA_flows


✅ olca-ipc and olca-schema imported successfully


## Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results 

In [ ]:
# Run PrOMMiS model and extract data
prommis_data = prommis_LCA_data.main()

## Step 3: Review PrOMMiS Results

In [3]:
prommis_data.head()

,Flow,Source,In/Out,Category,Value 1,Unit 1,Value 2,Unit 2
0,Inerts,Solid Feed,In,Solid Input,22.68,kg/hr,0.6952,mass fraction
1,Scandium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000028,mass fraction
2,Yttrium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000033,mass fraction
3,Lanthanum Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000068,mass fraction
4,Cerium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000156,mass fraction


## Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows

In [4]:
# Convert PrOMMiS Data in prommis_data dataframe to LCA-relevant units
df = prommis_data
prommis_LCA_conversions.main()
# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

2025-08-01 14:14:03,031 - WARNING - Dynamically registering the following numeric type:
    pymatgen.core.units.FloatWithUnit
Dynamic registration is supported for convenience, but there are known
limitations to this approach.  We recommend explicitly registering
numeric types using RegisterNumericType() or RegisterIntegerType().


,Unnamed: 0,Flow,Source,In/Out,Category,Value 1,Unit 1,Value 2,Unit 2,LCA Amount,LCA Unit
0,0,Inerts,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.695200,mass fraction,1.576714e+01,kg
1,1,Scandium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000028,mass fraction,6.304269e-04,kg
2,2,Yttrium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000033,mass fraction,7.453850e-04,kg
3,3,Lanthanum Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000068,mass fraction,1.537180e-03,kg
4,4,Cerium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000156,mass fraction,3.541731e-03,kg
5,5,Praseodymium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000017,mass fraction,3.888214e-04,kg
6,6,Neodymium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000068,mass fraction,1.534570e-03,kg
7,7,Samarium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000015,mass fraction,3.354962e-04,kg
8,8,Gadolinium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000010,mass fraction,2.359854e-04,kg
9,9,Dysprosium Oxide,Solid Feed,In,Solid Input,2.268000e+01,kg/hr,0.000008,mass fraction,1.711948e-04,kg


## Step 6: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows

In [5]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit

finalize_LCA_flows.main()

=== Finalize DataFrame Workflow ===

Converted LCA DataFrame Head:
   Unnamed: 0             Flow      Source In/Out     Category  Value 1  \
0           0           Inerts  Solid Feed     In  Solid Input    22.68   
1           1   Scandium Oxide  Solid Feed     In  Solid Input    22.68   
2           2    Yttrium Oxide  Solid Feed     In  Solid Input    22.68   
3           3  Lanthanum Oxide  Solid Feed     In  Solid Input    22.68   
4           4     Cerium Oxide  Solid Feed     In  Solid Input    22.68   

  Unit 1   Value 2         Unit 2  LCA Amount LCA Unit  
0  kg/hr  0.695200  mass fraction   15.767136       kg  
1  kg/hr  0.000028  mass fraction    0.000630       kg  
2  kg/hr  0.000033  mass fraction    0.000745       kg  
3  kg/hr  0.000068  mass fraction    0.001537       kg  
4  kg/hr  0.000156  mass fraction    0.003542       kg  


Applied functional unit conversion with scaling factor: 1.596528895384177e-05
Reference flow: 99.85% REO Product from Roaster Product
Fina

,Flow_Name,LCA_Amount,LCA_Unit,Is_Input,Reference_Product,Flow_Type,Description
0,374 ppm REO Feed,1.420741e+06,kg,True,False,Solid Input,
1,99.85% REO Product,1.000000e+00,kg,False,True,Solid Output,
2,Carbon Dioxide,9.283150e-01,kg,False,False,Emissions to air,
3,Cleaner Circuit Purge,3.884051e+02,m3,False,False,Wastewater,
4,DEHPA,7.580114e+05,kg,True,False,Chemicals,
5,Dust and Volatiles,7.359133e-02,kg,False,False,Solid Waste,
6,Electricity,1.628458e+03,kWh,True,False,Electricity,
7,Filter Cake,1.392959e+06,kg,False,False,Solid Waste,
8,Heat,1.629930e+05,MJ,True,False,Heat,
9,Hydrochloric Acid,3.754395e+03,kg,True,False,Chemicals,


## Step 7: Connect to openLCA

### Notes on modeling this process using openLCA

Before running the following code:
- Open the software openLCA
- Open an existing database or create a new one
- Import libraries with existing processes
- Use the IPC server from the Developer Tools
- Run the server on port 8080 

In [4]:
print("\n🔌 STEP 1: Connecting to openLCA...")
client = initialize_client()

2025-08-02 13:36:04,591 - INFO - ✅ Connection test successful. Found 1957 processes.



🔌 STEP 1: Connecting to openLCA...
✅ Connected to openLCA


#### Define dataframe to be used

The dataframe should have the following columns: 

| Flow_Name | LCA_Unit | LCA_Amount | Is_Input | Reference_Product | Flow_Type | Description |

In [5]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('lca_df_finalized.csv')


#### Enter Process Information
##### Unit Process Name

In [6]:
process_name = "Test Process 1"  # Replace with the actual process name you want to use

#### Unit Process Description 

In [7]:
process_description = " Example Process Description"  # Replace with the actual process description you want to use

#### Enter Unit Process Flows

In [8]:
process = create_process_from_dataframe_with_selection(
    client=client,
    df=df,
    process_name=process_name,
    process_description=process_description
)



🔍 Processing 23 flows for process: Test Process 1

📋 Row 1: 374 ppm REO Feed
   Unit: kg, Amount: 1420741.1877343715
   Type: Input

🔍 Search Options for Flow: 374 ppm REO Feed
💡 Choose search strategy:
   1. Search in PROCESSES (e.g., 'Coal power plant', 'Steel production')
   2. Search in FLOWS (e.g., 'CO2', 'Water', 'Electricity')
   3. Skip this flow

💡 Examples:
   - For 'Coal': Choose PROCESSES (search for coal power plants)
   - For 'CO2': Choose FLOWS (search for CO2 emissions flows)
   - For 'Water': Choose FLOWS (search for water flows)

🔍 Enter search keywords for PROCESS search:
Default would be: '374 ppm REO Feed'
   🔍 Searching for processes containing '374 ppm REO Feed'...
   🔍 Searching through 1957 processes...
   ❌ No processes found matching '374 ppm REO Feed' in process names
   📊 Found 0 matching processes

=== Process Selection for Flow: 374 ppm REO Feed ===
No existing processs found matching '374 ppm REO Feed'
Options:
1. Create new elementary flow
2. Create ne

AttributeError: 'Ref' object has no attribute 'flow_properties'

## Step 8: Create Product System

#### Here we will use the unit process created in the previous step to create a product system
#### This step uses the netlolca library to retrieve the process id and use it to build the product system using default providers

In [ ]:
# get the process id 
pid = process.id

# build supply chain using default providers
# pl_list: list of process links
# pd_list: list of process ids
pl_list, pd_list = build_supply_chain(pid)

# creating the product system structure will require the:
# Reference process
# the linking configuration
p_ref = client.query(olca.Process, pid).to_ref()
p_lc = olca.LinkingConfig()

# create the product system using openLCA client
ps_ref = client.create_product_system(p_ref, p_lc)

ps_obj = client.query(olca.ProductSystem, ps_ref.id)

# Add all processes from the supply chain
ps_obj.processes = [
    client.query(olca.Process, x).to_ref() for x in pd_list
]

# Add the process links
ps_obj.process_links = pl_list

# Add the completed product system to the database
client.add (ps_obj)
